# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [21]:
# Write your code below.
%load_ext dotenv
%dotenv 


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [22]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [27]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet"))


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [30]:
print(dd_px.columns)


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits', 'year'],
      dtype='object')


In [32]:
# Write your code below.
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
dd_feat = (dd_px
.assign(Adj_Close=lambda x: x['Close'].shift(1),
        Adj_Close_lag=lambda x: x['Adj_Close'].shift(1),
        returns=lambda x: (x['Adj_Close'] / x['Adj_Close_lag']) - 1,
        hi_lo_range=lambda x: x['High'] - x['Low']))

dd_feat.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,Adj_Close,Adj_Close_lag,returns,hi_lo_range
ticker,,,,,,,,,,,,,
A,2024-01-02 00:00:00-05:00,137.491417,139.879279,137.212834,138.048584,1441600,0.0,0.0,2024,NaN,NaN,NaN,2.666445
A,2024-01-03 00:00:00-05:00,137.302382,137.302382,130.407422,130.496964,2074500,0.0,0.0,2024,138.048584,NaN,NaN,6.894960
A,2024-01-04 00:00:00-05:00,129.890062,130.835256,129.531881,130.337784,2446600,0.0,0.0,2024,130.496964,138.048584,-0.054703,1.303375
A,2024-01-05 00:00:00-05:00,129.342812,131.292910,127.969783,129.899979,1394000,0.0,0.0,2024,130.337784,130.496964,-0.001220,3.323127
A,2024-01-08 00:00:00-05:00,129.482108,132.894776,129.153774,132.705734,1311400,0.0,0.0,2024,129.899979,130.337784,-0.003359,3.741002


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [36]:
# Write your code below.
df_feat = dd_feat.compute()
df_feat['rolling_avgerage_return'] = df_feat['returns'].rolling(10).mean()

df_feat.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,Adj_Close,Adj_Close_lag,returns,hi_lo_range,rolling_avgerage_return
ticker,,,,,,,,,,,,,,
A,2024-01-02 00:00:00-05:00,137.491417,139.879279,137.212834,138.048584,1441600,0.0,0.0,2024,NaN,NaN,NaN,2.666445,NaN
A,2024-01-03 00:00:00-05:00,137.302382,137.302382,130.407422,130.496964,2074500,0.0,0.0,2024,138.048584,NaN,NaN,6.894960,NaN
A,2024-01-04 00:00:00-05:00,129.890062,130.835256,129.531881,130.337784,2446600,0.0,0.0,2024,130.496964,138.048584,-0.054703,1.303375,NaN
A,2024-01-05 00:00:00-05:00,129.342812,131.292910,127.969783,129.899979,1394000,0.0,0.0,2024,130.337784,130.496964,-0.001220,3.323127,NaN
A,2024-01-08 00:00:00-05:00,129.482108,132.894776,129.153774,132.705734,1311400,0.0,0.0,2024,129.899979,130.337784,-0.003359,3.741002,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

No. Dask also has rolling function similar to pandas.
+ Would it have been better to do it in Dask? Why?

Yes. Dask is more efficient for large dataset.
(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.